## Project Title

### Project Description

Our team will focus on studying the relationship (if any) of the average land temperature (i.e., #2 proposed dataset) and the evolution of population in such land, along with its GDP. Datasets will be pulled from the World Bank and are available in `.csv`, `.xml` and `.xlsx` formats in the following links, respectively:

[Population](https://data.worldbank.org/indicator/SP.POP.TOTL)

[GDP (2015 USD)](https://data.worldbank.org/indicator/NY.GDP.MKTP.KD)

Behavior will be analysed mainly by visual means (such as scatterplots, pyramid populations, etc.) and reported in Markdown in the corresponding .ipynb file. 

> Imported libraries

In [8]:
import pandas as pd
import geopandas
import folium 

## Programming Component


### Reading and cleaning the main dataset


In [21]:
df = pd.read_csv('..\data\land-data\GlobalLandTemperaturesByCountry.csv') # Reading GlobalLandTemperaturesByCountry csv file
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


Next setp will be convert ´dt´ colum in to a date type one and we will only be focusing in the ´year´ information. 

In [22]:
df.dt = pd.to_datetime(df.dt) # Convert to datetime
df.dt = df.dt.dt.year # Extract year

In [23]:
df.rename(columns={"dt": "Year"}, inplace=True) # Rename column
df_year_temp =df.groupby(['Country', 'Year']).mean() # Group by country and year
df_year_temp.drop(columns=['AverageTemperatureUncertainty'], inplace=True) # Drop unnecessary column
df_year_temp.head() 

AverageTemperature
Country     Year                    
Afghanistan 1838           18.379571
            1839                 NaN
            1840           13.413455
            1841           13.997600
            1842           15.154667

In [24]:
df_year_temp.reset_index(inplace=True) # Let's reset the index so that we can use the Country and Year columns as a key to merge with the other datasets
df_year_temp.head()

,Country,Year,AverageTemperature
0,Afghanistan,1838,18.379571
1,Afghanistan,1839,NaN
2,Afghanistan,1840,13.413455
3,Afghanistan,1841,13.997600
4,Afghanistan,1842,15.154667


### Reading and cleaning the 2 others data sets

In [27]:
df_gdp = pd.read_csv('..\data\gdp-data\GDP_Country.csv', skiprows=4) # Reading GDP_Country csv file
df_gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.862306e+09,2.861720e+09,2.963128e+09,3.025850e+09,3.191738e+09,3.359555e+09,3.380889e+09,2.752412e+09,3.225070e+09,NaN
1,Africa Eastern and Southern,AFE,GDP (constant 2015 US$),NY.GDP.MKTP.KD,1.536964e+11,1.540611e+11,1.663621e+11,1.749528e+11,1.829721e+11,1.927209e+11,...,8.623341e+11,8.971645e+11,9.231439e+11,9.460928e+11,9.710653e+11,9.964178e+11,1.016728e+12,9.857923e+11,1.029191e+12,NaN
2,Afghanistan,AFG,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.918925e+10,1.971207e+10,1.999816e+10,2.045018e+10,2.099149e+10,2.124113e+10,2.207200e+10,2.155306e+10,1.708357e+10,NaN
3,Africa Western and Central,AFW,GDP (constant 2015 US$),NY.GDP.MKTP.KD,1.056755e+11,1.076147e+11,1.116749e+11,1.198082e+11,1.262691e+11,1.313913e+11,...,7.046760e+11,7.464664e+11,7.669580e+11,7.678299e+11,7.855332e+11,8.086763e+11,8.344802e+11,8.269667e+11,8.597592e+11,NaN
4,Angola,AGO,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.243377e+10,8.640707e+10,8.721930e+10,8.496904e+10,8.484159e+10,8.372481e+10,8.313874e+10,7.848297e+10,7.934628e+10,NaN



### Combining and cleaning data



### Calculations